# Geospatial API Project
Silver Sloths - Habeeb Branch<br>
October 2, 2021

In [1]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json
from io import StringIO
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
# Read in all aggravated burglary data between January 1, 2021 and June 30, 2021
burglary_endpoint =  "https://data.nashville.gov/resource/2u6v-ujjs.geojson"
params = {
    "$where": "incident_reported between '2021-01-01T00:00:00' and '2021-06-30T23:59:59'",
    'offense_description': 'BURGLARY- AGGRAVATED',
    '$limit': '1000000'
}
response = requests.get(burglary_endpoint,  
                        params = params)
burglary = gpd.read_file(StringIO(response.text))

In [9]:
# Drop duplicate incidents
burglary = burglary.drop_duplicates(subset='incident_number')
burglary

,victim_county_resident,zip_code,victim_number,offense_nibrs,rpa,latitude,victim_race,incident_number,investigation_status,offense_number,...,weapon_primary,report_type_description,victim_type,incident_status_code,incident_status_description,zone,victim_gender,incident_occurred,primary_key,geometry
0,RESIDENT,None,1,220,8203,36.15,B,20210249540,Open,1,...,09,DISPATCHED,I,O,OPEN,511,F,2021-05-04T23:45:00,20210249540_11,POINT (-86.77000 36.15000)
1,RESIDENT,None,1,220,9615,36.2,W,20210248840,Open,1,...,09,DISPATCHED,I,O,OPEN,525,F,2021-05-04T12:06:00,20210248840_11,POINT (-86.63000 36.20000)
2,NON RESIDENT,None,1,220,6901,36.14,W,20210244919,Open,1,...,17,DISPATCHED,I,O,OPEN,815,M,2021-05-01T23:00:00,20210244919_11,POINT (-86.78000 36.14000)
3,NON RESIDENT,None,3,220,None,36.14,W,20210244892,Open,1,...,17,DISPATCHED,I,O,OPEN,None,M,2021-05-01T21:30:00,20210244892_13,POINT (-86.78000 36.14000)
5,RESIDENT,None,2,220,8401,36.11,W,20210248402,Open,1,...,17,DISPATCHED,I,O,OPEN,313,F,2021-05-04T00:00:00,20210248402_12,POINT (-86.73000 36.11000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,RESIDENT,37013,1,220,8861,36.052,B,20210029313,Closed,1,...,01,DISPATCHED,I,A,CLEARED BY ARREST,327,F,2021-01-15T21:50:00,20210029313_11,POINT (-86.64700 36.05200)
1047,RESIDENT,37210,1,220,8201,36.152,B,20210073787,Closed,1,...,09,DISPATCHED,I,A,CLEARED BY ARREST,511,F,2021-02-06T12:00:00,20210073787_11,POINT (-86.76500 36.15200)
1048,RESIDENT,37207,1,220,3013,36.204,B,20210127915,Closed,1,...,17,DISPATCHED,I,A,CLEARED BY ARREST,631,M,2021-03-03T15:30:00,20210127915_11,POINT (-86.80700 36.20400)
1049,None,None,1,220,8655,36.05,None,20210126551,Open,1,...,17,None,B,O,OPEN,827,None,2021-02-23T10:30:00,20210126551_11,POINT (-86.72000 36.05000)


In [4]:
# Read in census tract shape files
census_tract = gpd.read_file('../data/tl_2019_47_tract.shp')
burglary.crs = census_tract.crs
burglary_in_census_tract = gpd.sjoin(burglary, census_tract, op='within')
burglary_in_census_tract['NAMELSAD'].value_counts()

Census Tract 163       53
Census Tract 162       21
Census Tract 110.01    20
Census Tract 195       19
Census Tract 193       19
                       ..
Census Tract 179.02     1
Census Tract 103.03     1
Census Tract 178        1
Census Tract 132.02     1
Census Tract 110.02     1
Name: NAMELSAD, Length: 148, dtype: int64

Census tract 163 has the most aggravated burglary cases between Janury 1, 2021 and June 30, 2021 at 53 cases.

In [5]:
# Read in API key for census site
with open('../data/census.json') as fi:
    credentials = json.load(fi)
    
api_key = credentials['api_key']

Data read in via an API should be here.